In [17]:
%reload_ext autoreload
%autoreload 2

In [18]:
from result_saver import SaverProvider

provider = SaverProvider()

# Parameters

In [19]:
# Fixed parameters

DEVICE = 'ibm_sherbrooke'
_RESETS = False

backend = provider.get_backend(DEVICE)

In [20]:
# Variable parameters
import numpy as np

logicals = ['0', '1']
x_basis = [True, False]

rounds = [10, 20, 30, 40, 50, 75, 100]


print("rounds:", rounds)


rounds: [10, 20, 30, 40, 50, 75, 100]


# Find the best layout

In [21]:
from soft_info import BackendEvaluator

evaluator = BackendEvaluator(backend)
longest_path, _, _, path_info = evaluator.find_longest_good_RepCode_string(plot=False, Ancilla_threshold=0.3)

Trying RepCode string of length 109 => distance 55... Current maxs: CX: 1.0, Ancilla: 1.0
Trying RepCode string of length 107 => distance 54... Current maxs: CX: 1.0, Ancilla: 0.45519999999999994
Trying RepCode string of length 105 => distance 53... Current maxs: CX: 1.0, Ancilla: 0.07666666666666666
Trying RepCode string of length 103 => distance 52... Current maxs: CX: 0.032330579322472325, Ancilla: 0.45519999999999994


In [22]:
path_info

{'mean_gate_error': 0.009235693054578803,
 'min_gate_error': 0.0032001821523091334,
 'max_gate_error': 0.032330579322472325,
 'mean_readout_error': 0.022919417475728152,
 'min_readout_error': 0.0025999999999999357,
 'max_readout_error': 0.45519999999999994,
 'mean_ancilla_error': 0.019960784313725485,
 'min_ancilla_error': 0.0025999999999999357,
 'max_ancilla_error': 0.2714,
 'CX_fidelity': 0.3874641551835346}

In [23]:
link_qubits = longest_path[1::2]
code_qubits = longest_path[0::2]
layout = link_qubits + code_qubits
distance = len(code_qubits)
print("longest path:", longest_path)
print("layout:", layout)
print("distance:", distance)

longest path: [100, 99, 98, 91, 79, 80, 81, 72, 62, 61, 60, 59, 58, 71, 77, 76, 75, 90, 94, 95, 96, 109, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 112, 108, 107, 106, 105, 104, 103, 102, 92, 83, 84, 85, 86, 87, 88, 89, 74, 70, 69, 68, 67, 66, 65, 64, 54, 45, 44, 43, 42, 41, 40, 39, 33, 20, 19, 18, 14, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 17, 30, 31, 32, 36, 51, 50, 49, 48, 47, 35, 28, 27, 26, 25, 24, 23, 22]
layout: [99, 91, 80, 72, 61, 59, 71, 76, 90, 95, 109, 115, 117, 119, 121, 123, 125, 112, 107, 105, 103, 92, 84, 86, 88, 74, 69, 67, 65, 54, 44, 42, 40, 33, 19, 14, 1, 3, 5, 7, 9, 11, 17, 31, 36, 50, 48, 35, 27, 25, 23, 100, 98, 79, 81, 62, 60, 58, 77, 75, 94, 96, 114, 116, 118, 120, 122, 124, 126, 108, 106, 104, 102, 83, 85, 87, 89, 70, 68, 66, 64, 45, 43, 41, 39, 20, 18, 0, 2, 4, 6, 8, 10, 12, 30, 32, 51, 49, 47, 28, 26, 24, 22]
distance: 52


# No rounds loop

In [24]:
from qiskit_qec.circuits import RepetitionCodeCircuit
from soft_info import run_IQ_calibration
from qiskit import transpile
from Scratch import metadata_helper
from tqdm import tqdm


############################

# Round index
round_index = 5

# Z = 0
xbasis = False
logical = '0'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=4*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1) 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'Subset {distance}, {T} rounds, {logical} log, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)


Submitting for rounds =  75


100%|██████████| 20/20 [05:04<00:00, 15.24s/it]


In [25]:
############################

# Z = 1
xbasis = False
logical = '1'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=4*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1) 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'Subset {distance}, {T} rounds, {logical} log, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)   

Submitting for rounds =  75


100%|██████████| 20/20 [04:03<00:00, 12.19s/it]


In [26]:
############################

# X = 0
xbasis = True
logical = '0'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=4*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1) 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'Subset {distance}, {T} rounds, {logical} log, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  75


100%|██████████| 20/20 [04:01<00:00, 12.06s/it]


In [27]:
############################

# X = 1
xbasis = True
logical = '1'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=4*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1) 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'Subset {distance}, {T} rounds, {logical} log, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  75


100%|██████████| 20/20 [03:50<00:00, 11.52s/it]


# Round loop!

In [30]:

# Z = 0
xbasis = False
logical = '0'

# Run calibration

for T in rounds:
    code = RepetitionCodeCircuit(distance, T, xbasis=xbasis)
    qc = code.circuit[logical]
    transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1) 

    n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job
    metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                               distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                               path_info=path_info)

    # backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
    #             job_tags=[f'Subset {distance}, {T} rounds, {logical} log, xbasis={xbasis}'])

# Run calibration
    

{'descr': 'subset RepCodes', 'code': 'RepetitionCodeCircuit', 'distance': 52, 'rounds': 10, 'logical': '0', 'xbasis': False, 'path_info': {'mean_gate_error': 0.007963147793478936, 'min_gate_error': 0.0034766228550469347, 'max_gate_error': 0.0232547718972346, 'mean_readout_error': 0.022770873786407766, 'min_readout_error': 0.0019000000000000128, 'max_readout_error': 0.3568, 'mean_ancilla_error': 0.02186078431372549, 'min_ancilla_error': 0.0019000000000000128, 'max_ancilla_error': 0.24750000000000005}}
{'descr': 'subset RepCodes', 'code': 'RepetitionCodeCircuit', 'distance': 52, 'rounds': 20, 'logical': '0', 'xbasis': False, 'path_info': {'mean_gate_error': 0.007963147793478936, 'min_gate_error': 0.0034766228550469347, 'max_gate_error': 0.0232547718972346, 'mean_readout_error': 0.022770873786407766, 'min_readout_error': 0.0019000000000000128, 'max_readout_error': 0.3568, 'mean_ancilla_error': 0.02186078431372549, 'min_ancilla_error': 0.0019000000000000128, 'max_ancilla_error': 0.24750000